In [1]:
## Анализ данных

### Импорт необходимых библиотек

In [5]:
import pandas as pd
import numpy as np
from scipy import stats as st
import matplotlib.pyplot as plt
import statsmodels.stats.multitest as smt

Нейминг испытуемых:

* Фамилия 1 - испытуемый № 1
* Фамилия 2 - испытуемый № 2
* Фамилия 3 - испытуемый № 3
* Фамилия 4 - испытуемый № 4

In [6]:
def naming(name_name):
    if name_name == 'Фамилия 1':
        return 'испытуемый № 1'
    elif name_name == 'Фамилия 2':
        return 'испытуемый № 2'
    elif name_name == 'Фамилия 3':
        return 'испытуемый № 3'
    elif name_name == 'Фамилия 4':
        return 'испытуемый № 4'
    else:
        return 'error name'

### Выбор файлов для первичного анализа

In [3]:
files_path = ['список файлов']

### Создание функции для первичного анализа

In [4]:
def prepare(files_path):
    col_names = ['список названий столбцов входных данных']
    
    col_names_out = ['список названий выходных данных']
    
    data = []
    
    # перебор файлов и страниц в файлах
    for file_path in files_path:
        for number_sheet in range(0, 8):
            # считывание информации
            df_info = pd.read_excel(file_path, sheet_name=number_sheet, nrows=5, usecols="B:C", names=['name', 'value'])
            df = pd.read_excel(file_path, sheet_name=number_sheet, names=col_names, skiprows=10, usecols="B:BT")            
            #выделение нужного участка
            df = df[~df['time'].isna()]            
            #добавление лага по времени
            df['delta_lead_time'] = df['time'].shift(-1) - df['time']
            
            #расчет метрик
            #metric_1
            metric_1 = np.round(df[df['param'] != 0].reset_index()['time'][0], 2) if df[df['param'] != 0].shape[0] > 0 else np.NaN
            #metric_2
            metric_2 = np.round(df[df['param'] != 0].reset_index()['time'][0], 2) if df[df['param'] != 0].shape[0] > 0 else np.NaN
            #metric_3
            if df[df['param'] == 9].shape[0] > 0:
                metric_3 = np.round(df[df['param'] == 9].reset_index()['time'][0], 2)
                metric_3_dop = df[df['time'] < metric_2].query('условия').shape[0]*0.02
                metric_3_dop = df[df['time'] < metric_2].query('условия').shape[0]*0.02
                metric_3_dop = df[df['time'] < metric_2].query('условия').shape[0]*0.02
                metric_3_dop = df[df['time'] < metric_2].query('условия').shape[0]*0.02
            else:
                metric_3 = np.NaN
                metric_3_dop = np.NaN
                metric_3_dop = np.NaN
                metric_3_dop = np.NaN
                metric_3_dop = np.NaN
            #metric_4
            metric_4 = np.round(df[(df['time'] < metric_2) & (df['param'] == 1)]['param'].sum(), 2)
            #и еще многие метрики

            #добавление расчитанных метрик к списку
            data.append([df_info['value'][2], df_info['value'][3], df_info['value'][4], df_info['value'][0], df_info['value'][1],
                         metric_1, metric_2, metric_3, metric_4])
    
    #возвращаем датафрейм
    return pd.DataFrame(data = data, columns=col_names_out)

### Получение выходного файла

In [ ]:
%%time
df_out = prepare(files_path)

Контроль пропусков значений в параметрах ...

In [ ]:
df_out[df_out['time_pusk'].isna() | df_out['time_to_zahvat'].isna() | df_out['time_PR'].isna() | df_out['time_to_4pp_up'].isna() | df_out['time_to_one_click_vvod'].isna() | df_out['time_first_ud'].isna()| df_out['count_drop_vvod_zahvat'].isna()]

## Построение дизайна исследования

Создадим словарь названий параметров

In [ ]:
dict_bp = {'metric_1':['название метрики на русском языке для вставки на графики'],
           'metric_2':['название метрики на русском языке для вставки на графики'],
           'metric_3':['название метрики на русском языке для вставки на графики'],
           'metric_4':['название метрики на русском языке для вставки на графики']}

Разделим датасет

In [ ]:
df_1 = df_out['filter']sort_values(['param']).reset_index(drop=True)
df_2 = df_out['filter']sort_values(['param']).reset_index(drop=True)

Сопоставим данные для применения парного статистического теста

In [ ]:
columns_equals = ['param equelity']
if df_1[columns_equals].equals(df_2[columns_equals]):
    print('Можно проводить тест!')
else:
    print('Необходимо привести данные к парному формату!!!')
    display(df_1[columns_equals])
    display(df_2[columns_equals])

Создадим функцию удаления пропущенных значений при сопоставлении

In [ ]:
def set_to_pair(s):
    # реализация содержала слишком много ссылок на именованные датасеты

Создадим функцию для проверки данных на нормальность распределения для определения статистического теста

In [ ]:
def norm_analisys(column_names):
    is_norm = []
    norm_pathes = []
    for s in column_names:
        df_1_pair, df_2_pair = set_to_pair(s)
        series_2 = df_2_pair[s]
        series_1 = df_1_pair[s]
        
        plt.figure(figsize=(10, 4), dpi=300)        
        p_val_2 = np.round(st.shapiro(series_2).pvalue, 3)
        plt.subplot(1, 2, 1)
        plt.hist(series_2, bins=10, density=True, alpha=0.6, color='b')
        xmin, xmax = plt.xlim()
        x = np.linspace(xmin, xmax, 100)
        mu, std = st.norm.fit(x)
        p = st.norm.pdf(x, mu, std)
        plt.plot(x, p, 'k', linewidth=2)
        plt.title(f'name (p_value={p_val_2})', fontsize=12)
        
        p_val_1 = np.round(st.shapiro(series_1).pvalue, 3)
        plt.subplot(1, 2, 2)
        plt.hist(series_1, bins=10, density=True, alpha=0.6, color='b')
        xmin, xmax = plt.xlim()
        x = np.linspace(xmin, xmax, 100)
        mu, std = st.norm.fit(x)
        p = st.norm.pdf(x, mu, std)
        plt.plot(x, p, 'k', linewidth=2)
        plt.title(f'name (p_value={p_val_1})', fontsize=12)
        
        title=''
        if min([p_val_2, p_val_1]) < 0.05:
            title = f'Параметр "{dict_bp[s][0]}" (ненормальное)'
            is_norm.append('нет')
        else:
            title = f'Параметр "{dict_bp[s][0]}" (нормальное)'
            is_norm.append('да')
            
        plt.suptitle(title, fontsize=14)
        path = 'images/norm/'+dict_bp[s][0]+'.png'
        plt.savefig(path)
        norm_pathes.append(path)
        plt.show()
        
    return pd.DataFrame(np.array(is_norm).reshape(1, len(column_names)), columns=column_names, index=['Распределение нормальное?']), norm_pathes

In [ ]:
df_norm, norm_pathes = norm_analisys(list(dict_bp.keys()))

Создадим функцию отрисовки столбчатых диаграмм

In [ ]:
def create_barplot(ax, df_1_pair, df_2_pair, s, norm, name):
    if name == 'all':
        series_1 = #
        series_2 = #
        stage_names = #
        title = 'среднее по всем'
        if norm:
            1_mean = np.round(np.mean(df_1_pair[s]), 2)
            2_mean = np.round(np.mean(df_2_pair[s]), 2)
        else:
            1_mean = np.round(np.median(df_1_pair[s]), 2)
            2_mean = np.round(np.median(df_2_pair[s]), 2)   
    else:
        series_1 = list(df_1_pair[df_1_pair['name']==name][s])
        series_2 = list(df_2_pair[df_2_pair['name']==name][s])
        stage_names = [str(s) for s in list(df_1_pair[df_1_pair['name']==name]['stage'].unique())]
        title = 'испытуемый '+ naming(name)
        if norm:
            1_mean = np.round(np.mean(series_1), 2)
            2_mean = np.round(np.mean(series_2), 2)
        else:
            1_mean = np.round(np.median(series_1), 2)
            2_mean = np.round(np.median(series_2), 2)
        
    bar1 = ax.bar(x=stage_names, height = series_1, alpha=0.5, color='green', label='name')
    bar2 = ax.bar(x=stage_names, height = series_2, alpha=0.5, color='blue', label='name')
    if name == 'all':
        ax.legend([bar1, bar2], ['name', 'name'])
    ax.axhline(1_mean, color='green', linestyle='dashed', label='name')
    ax.axhline(2_mean, color='blue', linestyle='dashed', label='name')
    ax.text(-0.042*(len(series_1)+10), 1_mean+0.1, f'{1_mean}', dict(fontsize=12, color='green'))
    ax.text(-0.042*(len(series_1)+10), 2_mean+0.1, f'{2_mean}', dict(fontsize=12, color='blue'))
    ax.set_title(title, fontsize=12)
    return np.round(1_mean - 2_mean, 2)

Применим статистический тест для парных выборок Стьюдента к параметрам для которых гипотеза о нормальности распределения не отвергнута, а для остальных тест Вилкоксона (в качестве поправки на множественные сравнения использована поправка Холма-Бонферрони)

In [ ]:
def count_adjuct_pvals(columns_names):
    #Расчет pvalue
    pvals = []
    for s in columns_names:
        df_1_pair, df_2_pair = set_to_pair(s)
        
        if df_norm[s][0] == 'да':
            pvals.append(np.round(st.ttest_rel(df_2_pair[s], df_1_pair[s]).pvalue, 3))
        else:
            x = df_2_pair[s] - df_1_pair[s]
            pvals.append(np.round(st.wilcoxon(x[x!=0]).pvalue, 3))
    
    #корректировка pvalue методом Хомла
    pvals_adj = {}
    p_adj = smt.multipletests(pvals, method='holm')[1]
    for i in range(len(columns_names)):
        pvals_adj[columns_names[i]] = np.round(p_adj[i], 3)
    
    texts = []
    pathes = []
    #Вывод текста и отображение графиков
    for s in columns_names:
        df_1_pair, df_2_pair = set_to_pair(s)
        
        pval = pvals_adj[s]
        text = ''

        #создадим картинку барплотов
        list_names = [''] #list(df_1_pair['name'].unique())
        fig, axes = plt.subplots(len(list_names)+1, 1, constrained_layout=True)
        fig.set_size_inches(10, 2.7*(len(list_names)+1))
        fig.set_dpi(300)
        difference = create_barplot(axes[0], df_1_pair, df_2_pair, s, df_norm[s][0], 'all')
        for i in range(len(list_names)):
            create_barplot(axes[1 + i], df_1_pair, df_2_pair, s, df_norm[s][0], list_names[i])
        fig.suptitle(f'Параметр {dict_bp[s][0]}', fontsize=20)
        fig.supxlabel('', fontsize=16)
        fig.supylabel(dict_bp[s][0] + ', '+dict_bp[s][1], fontsize=16)
        path = 'images/results/'+dict_bp[s][0]+'.png'
        #создадим текст
        if df_norm[s][0] == 'да':            
            if pval < 0.05:
                text = f'Разница средних значений параметра "{dict_bp[s][0]}" name и name {difference} {dict_bp[s][1]} достоверна (pvalue={pval})'
            else:
                text = f'Достоверных различий изменения параметра "{dict_bp[s][0]}" между name и name не обнаружено (p_value_adj={pval})'
        else:           
            if pval < 0.05:
                text = f'Разница медиан параметра "{dict_bp[s][0]}" name и name {difference} {dict_bp[s][1]} достоверна (pvalue={pval})'
            else:
                text = f'Достоверных различий изменения параметра "{dict_bp[s][0]}" между name и name не обнаружено (p_value_adj={pval})'
        
        print(text)
        texts.append(text)
        
        plt.savefig(path)
        pathes.append(path)
        plt.show()
        
    return pd.DataFrame(np.array([columns_names, pathes, list(pvals_adj.values()), texts]).T, columns=['value', 'path', 'p_value', 'text'])

In [ ]:
df_bp = count_adjuct_pvals(list(dict_bp.keys()))
df_bp = df_bp.astype({'p_value':'float'})

### Следующая часть анализа исследования (пердобработка)

In [ ]:
def uvl(file_path):
    for sheet_number in range(0, 8):
        try:
            # считывание файла информации
            df_info = pd.read_excel(file_path, sheet_name=sheet_number, nrows=5, usecols="B:C", names=['name', 'value'])
            name = #
            stage = #
            scenario = #
            # считывание файла
            df_uvl = # 
            # считывание файла метрик
            df_uvl_metrics = df_out[(df_out['name'] == name) & (df_out['stage'] == stage) & (df_out['scenario'] == scenario)].reset_index(drop=True)
            # выделение нужного участка и дальнейшая предобработка
        except:
            print(file_path + ' ' + str(stage))

In [ ]:
for path in files_path:
    uvl(path)

In [ ]:
def split_df_uvl(df):
    for i in range(df.shape[0]):
        df.loc[i, 'name'] = df['Файл'][i][:-15]
        df.loc[i, 'scenario'] = df['Файл'][i][-14:-7]
        df.loc[i, 'stage'] = df['Файл'][i][-6:-4]
    return df

In [ ]:
df_uvl = split_df_uvl(df_uvl).loc[:, 'rus_kren x':'stage']
df_uvl.head()

In [ ]:
df_uvl.to_csv('df_uvl_out.csv', sep='\t', encoding='utf-8')

### Следующая часть (основной цикл)

In [8]:
def set_to_pair_uvl(df):
    df_1 = df[df['scenario'] == 'СП_name'].sort_values(by=['name', 'stage'], ascending=False).reset_index(drop=True)
    df_2 = df[df['scenario'] == 'СП_name'].sort_values(by=['name', 'stage'], ascending=False).reset_index(drop=True)
    for i in range(int(df.shape[0]/2)):
        if not ((df_1.loc[i, 'name'] == df_2.loc[i, 'name']) & (df_1.loc[i, 'stage'] == df_2.loc[i, 'stage'])):
            print(df_1.loc[i, ['name', 'stage', 'scenario']])
            print(df_2.loc[i, ['name', 'stage', 'scenario']])
            return None
    return df_1, df_2

In [ ]:
df_1_uvl, df_2_uvl = set_to_pair_uvl(df_uvl)

columns_for_test = df_uvl.columns[1:27]


data_2 = np.round(np.mean(df_2_uvl[columns_for_test], axis=0),1)
data_1 = np.round(np.mean(df_1_uvl[columns_for_test], axis=0), 1)
difference_2_1 = data_2 - data_1
p_values = np.array([np.round(st.wilcoxon(x=df_1_uvl[column], y=df_2_uvl[column]).pvalue, 3) for column in columns_for_test]).T
p_val_series = pd.Series(p_values, index=columns_for_test)
df_p_uvl = pd.DataFrame([data_2, data_1, difference_2_1, p_val_series], columns=columns_for_test, 
                        index=['2', '1', 'difference_2_1', 'pvalue'])
df_p_uvl

In [13]:
df_p_uvl.to_excel('df_uvl_new_mean.xlsx')

### Следующая часть исследования

Создание функции построения круговых диаграмм

In [111]:
def func(pct, allvals):
    absolute = np.round(np.round(pct/100.*np.sum(allvals)), 2)
    return f"{pct:.1f}%\n({absolute:.1f} сек)"
        
def draw_circle_diagramm(df_1_pair, df_2_pair, name):
    s = 'metric'
    fig, axes = plt.subplots(1, 2)
    fig.set_dpi(300)
    radius = 0.0
    if name == 'all':
        title = f'Описание'  
        fig.set_size_inches(10, 5)
        radius = 1.2
    else:
        df_2_pair = df_2_pair[df_2_pair['name']==name]
        df_1_pair = df_1_pair[df_1_pair['name']==name]
        title = 'name '+ naming(name)
        fig.set_size_inches(10, 5)
        radius = 1.28

    fig.suptitle(title, fontsize=12)
    axes[0].set_xlabel(f'name', fontsize=12)
    axes[1].set_xlabel(f'name', fontsize=12)  
    
    data_2 = [np.round(df_2_pair['metric'].mean(), 2), 
                  np.round(df_2_pair['metric'].mean(), 2), 
                  np.round(df_2_pair['metric'].mean(), 2), 
                  np.round(df_2_pair['metric'].mean(), 2)]
    axes[0].pie(data_2, autopct=lambda pct: func(pct, data_2), textprops=dict(color="w", fontsize=12), 
                         colors=['b', 'y', 'g', 'r'], pctdistance=0.7, radius=radius)
    
    data_1 = [np.round(df_1_pair['metric'].mean(), 2), 
                 np.round(df_1_pair['metric'].mean(), 2),
                 np.round(df_1_pair['metric'].mean(), 2),
                 np.round(df_1_pair['metric'].mean(), 2)]
    axes[1].pie(data_1, autopct=lambda pct: func(pct, data_1), textprops=dict(color="w", fontsize=12), 
                         colors=['b', 'y', 'r', 'g'], pctdistance=0.7, radius=radius)
   
    legend_names = ['имена']
    fig.legend(legend_names, loc="lower center", fontsize="10")
    path = f'images/circles/{name}name.png'
    plt.savefig(path)
    plt.show()
    return path
    
def draw_all_circle_diagramms():
    df_1_pair, df_2_pair = set_to_pair('param')
    list_names = ['names'] #list(df_1_pair['name'].unique())
    pathes = []
    pathes.append(draw_circle_diagramm(df_1_pair, df_2_pair, 'all')) 
    for i in range(len(list_names)):
        pathes.append(draw_circle_diagramm(df_1_pair, df_2_pair, list_names[i]))
    return pathes

In [ ]:
circle_pathes = draw_all_circle_diagramms()

### Тут была интересная часть исследования по формированию таблицы с временами и цветами но из-за обилия демаскирующих данных удалил полностью раздел

## Формирование выходного ворд-файла

Функция-костыль для сброса нумерации списков

In [119]:
def list_number(doc, par, prev=None, level=None, num=True):
    """
    Makes a paragraph into a list item with a specific level and
    optional restart.

    An attempt will be made to retreive an abstract numbering style that
    corresponds to the style of the paragraph. If that is not possible,
    the default numbering or bullet style will be used based on the
    ``num`` parameter.

    Parameters
    ----------
    doc : docx.document.Document
        The document to add the list into.
    par : docx.paragraph.Paragraph
        The paragraph to turn into a list item.
    prev : docx.paragraph.Paragraph or None
        The previous paragraph in the list. If specified, the numbering
        and styles will be taken as a continuation of this paragraph.
        If omitted, a new numbering scheme will be started.
    level : int or None
        The level of the paragraph within the outline. If ``prev`` is
        set, defaults to the same level as in ``prev``. Otherwise,
        defaults to zero.
    num : bool
        If ``prev`` is :py:obj:`None` and the style of the paragraph
        does not correspond to an existing numbering style, this will
        determine wether or not the list will be numbered or bulleted.
        The result is not guaranteed, but is fairly safe for most Word
        templates.
    """
    xpath_options = {
        True: {'single': 'count(w:lvl)=1 and ', 'level': 0},
        False: {'single': '', 'level': level},
    }

    def style_xpath(prefer_single=True):
        """
        The style comes from the outer-scope variable ``par.style.name``.
        """
        style = par.style.style_id
        return (
            'w:abstractNum['
                '{single}w:lvl[@w:ilvl="{level}"]/w:pStyle[@w:val="{style}"]'
            ']/@w:abstractNumId'
        ).format(style=style, **xpath_options[prefer_single])

    def type_xpath(prefer_single=True):
        """
        The type is from the outer-scope variable ``num``.
        """
        type = 'decimal' if num else 'bullet'
        return (
            'w:abstractNum['
                '{single}w:lvl[@w:ilvl="{level}"]/w:numFmt[@w:val="{type}"]'
            ']/@w:abstractNumId'
        ).format(type=type, **xpath_options[prefer_single])

    def get_abstract_id():
        """
        Select as follows:

            1. Match single-level by style (get min ID)
            2. Match exact style and level (get min ID)
            3. Match single-level decimal/bullet types (get min ID)
            4. Match decimal/bullet in requested level (get min ID)
            3. 0
        """
        for fn in (style_xpath, type_xpath):
            for prefer_single in (True, False):
                xpath = fn(prefer_single)
                ids = numbering.xpath(xpath)
                if ids:
                    return min(int(x) for x in ids)
        return 0

    if (prev is None or
            prev._p.pPr is None or
            prev._p.pPr.numPr is None or
            prev._p.pPr.numPr.numId is None):
        if level is None:
            level = 0
        numbering = doc.part.numbering_part.numbering_definitions._numbering
        # Compute the abstract ID first by style, then by num
        anum = get_abstract_id()
        # Set the concrete numbering based on the abstract numbering ID
        num = numbering.add_num(anum)
        # Make sure to override the abstract continuation property
        num.add_lvlOverride(ilvl=level).add_startOverride(1)
        # Extract the newly-allocated concrete numbering ID
        num = num.numId
    else:
        if level is None:
            level = prev._p.pPr.numPr.ilvl.val
        # Get the previous concrete numbering ID
        num = prev._p.pPr.numPr.numId.val
    par._p.get_or_add_pPr().get_or_add_numPr().get_or_add_numId().val = num
    par._p.get_or_add_pPr().get_or_add_numPr().get_or_add_ilvl().val = level

Формирование файла

In [121]:
from docx import Document
from docx.shared import Inches
from docx.enum.style import WD_STYLE_TYPE
from docx.shared import Inches, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Mm

#создание документа
document = Document()

#задание формата А4
section = document.sections[0]
section = document.sections[0]
section.page_height = Mm(297)
section.page_width = Mm(210)
section.left_margin = Mm(30)
section.right_margin = Mm(15)
section.top_margin = Mm(20)
section.bottom_margin = Mm(20)

#настройка стиля
style = document.styles['Normal']
font = style.font
font.name = 'Tymes New Roman'
font.size = Pt(12)
style.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

#формирование текста
document.add_heading('Анализ данных', 0)

document.add_heading('Методика проведения исследований', level=1)

document.add_paragraph('Цель исследования: ', 'Normal')

document.add_paragraph('Задачи исследования:')

#список с применением функции костыля
p_list_1_1 = document.add_paragraph('Анализ 1', style='List Number')
p_list_1_2 = document.add_paragraph('Анализ 2', style='List Number')
p_list_1_3 = document.add_paragraph('Анализ 3', style='List Number')
p_list_1_4 = document.add_paragraph('Формирование выводов', style='List Number')
list_number(document, p_list_1_1)
list_number(document, p_list_1_2, prev=p_list_1_1)
list_number(document, p_list_1_3, prev=p_list_1_2)
list_number(document, p_list_1_4, prev=p_list_1_3)

#применение расчетных значений в тексте
n_names = len(df_out['name'].unique())
names = 'name' if n_names < 5 else 'name'
N = df_out.shape[0]
document.add_paragraph(f"В рамках данного исследования приняло участие...")

#список
document.add_paragraph('Порядок проведения статистического анализа:')
p_list_2_1 = document.add_paragraph('Сопоставление данных в целях проведения парного статистического теста (парных статистический тест имеет'+
                      ' большую мощность относительно непарного, а возможность обусловлена участием одних и тех же испытыемых'+
                      ' с применением идентичных...', style='List Number')
p_list_2_2 = document.add_paragraph('Анализ каждой переменной на нормальность распределения используя тест Шапиро-Уилка', style='List Number')
p_list_2_3 = document.add_paragraph('Выбор подходящего статистического теста (тест Стьюдента парных сравнений для'+
                      ' нормального распределения и тест Вилкоксона для ненормального распределения', style='List Number')
p_list_2_4 = document.add_paragraph('Введение поправки на множественные сравнения', style='List Number')
p_list_2_5 = document.add_paragraph('Визуализация полученных результатов (построение столбчатых диаграмм с указанием'+
                                   ' скорректированного p-value)', style='List Number')
p_list_2_6 = document.add_paragraph('Анализ полученных результатов и формирование выводов', style='List Number')
list_number(document, p_list_2_1)
list_number(document, p_list_2_2, prev=p_list_2_1)
list_number(document, p_list_2_3, prev=p_list_2_2)
list_number(document, p_list_2_4, prev=p_list_2_3)
list_number(document, p_list_2_5, prev=p_list_2_4)
list_number(document, p_list_2_6, prev=p_list_2_5)

document.add_paragraph('Используемые методы и инструменты:')
document.add_paragraph('Язык программирования Python - для проведения статистического анализа', style='List Bullet')
document.add_page_break()

document.add_heading('Результаты исследований', level=1)
document.add_heading('Анализ 1', level=2)
document.add_heading('Анализ 2', level=3)

for norm_path in norm_pathes:
    document.add_picture(norm_path, width=Mm(170))
document.add_page_break()

#вставка таблицы нормальности распределения
table_norm = document.add_table(rows=len(df_norm.columns)+1, cols=2)
row_name = table_norm.rows[0].cells
row_name[0].text = 'Название параметра'
row_name[1].text = 'Распределение параметра нормальное?'
p = row_name[1].paragraphs[0]
p.alignment = WD_ALIGN_PARAGRAPH.CENTER

for i in range(len(df_norm.columns)):
    row_name = table_norm.rows[i+1].cells
    row_name[0].text = dict_bp[df_norm.columns[i]][0]
    row_name[1].text = df_norm[df_norm.columns[i]][0]
    p = row_name[1].paragraphs[0]
    p.alignment = WD_ALIGN_PARAGRAPH.CENTER

table_norm.style = 'Light List Accent 1'

for row in table_norm.rows:
    for cell in row.cells:
        paragraphs = cell.paragraphs
        for paragraph in paragraphs:
            for run in paragraph.runs:
                font = run.font
                font.size= Pt(12)

document.add_paragraph('Как видно из результатов анализа распределение большинства переменных отлично от нормального и в соответствии с '+
                       'методикой к ним будет применен непараметрический статистический метод для парных выборок Вилкоксона. В случае '+
                       'переменной "максимальная перегрузка" будет применен параметрический тест, который обладает большей мощностью в '+
                       'сравнении с непараметрический аналогом, тест Стьюдента для парных выборок.', style='List Bullet')
document.add_page_break()

document.add_heading('Визуализация резуальтатов исследований', level=3)
for i in range(df_bp.shape[0]):
    document.add_picture(df_bp.loc[i, 'path'], width=Mm(170))
    document.add_paragraph(df_bp.loc[i, 'text'])
    
df_bp_dost = df_bp[df_bp['p_value'] < 0.05][['text', 'p_value']].reset_index(drop=True)

document.add_page_break()
if df_bp_dost.shape[0] > 0:
    table_bp = document.add_table(rows=df_bp_dost.shape[0]+1, cols=1)
    row_name = table_bp.rows[0].cells
    row_name[0].text = 'Достоверные отличия'
    p = row_name[0].paragraphs[0]
    p.alignment = WD_ALIGN_PARAGRAPH.CENTER

    for i in range(df_bp_dost.shape[0]):
        row_value = table_bp.rows[i+1].cells
        row_value[0].text = df_bp_dost.loc[i, 'text']

    table_bp.autofit = True
    table_bp.style = 'Colorful List Accent 4'
else:
    'Достоверных отличий анализируемых параметров не обнаружено'

document.add_paragraph('По результатам анализа выбранных метрик...', 
                       style='List Bullet')
document.add_page_break()  

document.add_heading('Анализ 2', level=2)
table_uvl = document.add_table(rows=3, cols=len(df_p_uvl.columns))

row_0 = table_uvl.rows[0].cells
row_1 = table_uvl.rows[1].cells
row_2 = table_uvl.rows[2].cells

row_0[0].text = ''
row_1[0].text = 'Разница (name - name)'
row_2[0].text = 'p value'

for i in range(1, df_p_uvl.shape[1]):
    row_0[i].text = df_p_uvl.columns[i]
    row_1[i].text = str(round(df_p_uvl.loc['difference_2_1', df_p_uvl.columns[i]], 1))
    row_2[i].text = str(round(df_p_uvl.loc['pvalue', df_p_uvl.columns[i]], 1))

col_name = table_uvl.columns[0].cells[0]
col_name.width = Mm(22)
table_uvl.autofit = False
table_uvl.style = 'Light List Accent 1'

document.add_paragraph('По результатам анализа ...', 
                       style='List Bullet')

document.add_page_break()  

document.add_heading('Анализ 3', level=2)
for circle_path in circle_pathes:
    document.add_picture(circle_path, width=Mm(170))
document.add_page_break()  

# вставки удаленных таблиц

document.add_page_break()
document.add_heading('Выводы по анализу...:', level=2)
document.add_paragraph('Метрики качества ...', 
                       style='List Bullet')
document.add_page_break()

document.save('report.docx')